In [1]:
import pandas
from faker import Factory
import random
from time import monotonic
import datetime
import os
from time import sleep
import json
from multiprocessing import Process
from tqdm import tqdm

from kafka import KafkaProducer


food_table = pandas.read_csv("data/starbucks.csv").values
faker = Factory.create()
payment = 'cash, credit, debit'.split(',')

names = ["Alicia", "Alex", "Julian", "Cassandra", "Mark", "Denver", "Luiza"]

discount = [10, 20, 30]


date = datetime.datetime(2020,8,1,12,4,5)
    
def create_random_event():
    name , size, price =random.choice(food_table)
    global date
    date += datetime.timedelta(minutes=60)
    index = random.choice(range(len(names)))
    
    return {'name_employer': names[index],
            'id_employer': index,
            'name_client': (faker.first_name() + " " + faker.last_name()),  
            'transaction_id': faker.ean(), 
            'payment': random.choice(payment),  
            'timestamp': str(date),
            'product_name': name,  
            'product_size': size,  
            'product_price': price,  
            'percent_discount': random.choice(discount),
           }

"""Produce fake transactions into a Kafka topic."""


TRANSACTIONS_TOPIC = "queueing.transactions"
KAFKA_BROKER_URL = "kafka:29092"
TRANSACTIONS_PER_SECOND = 100
SLEEP_TIME = 1 / TRANSACTIONS_PER_SECOND


producer = KafkaProducer(
    bootstrap_servers=KAFKA_BROKER_URL,
    # Encode all values as JSON
    value_serializer=lambda value: json.dumps(value).encode(),
)

for _ in tqdm(range(123)):
    transaction: dict = create_random_event()
    producer.send(TRANSACTIONS_TOPIC, value=transaction)
#     print(transaction)  # DEBUG
    sleep(SLEEP_TIME)

100%|██████████| 123/123 [00:01<00:00, 71.73it/s]


In [2]:
percent_discount = 10
((100 - percent_discount)/100)

0.9

In [3]:
food_table = pandas.read_csv("data/starbucks.csv")
food_table

,name,size,price
0,White Chocolate Mocha,Tall,3.75
1,White Chocolate Mocha,Grande,4.45
2,White Chocolate Mocha,Venti,4.75
3,Cinnamon Dolce Latte,Tall,3.65
4,Cinnamon Dolce Latte,Grande,4.25
5,Cinnamon Dolce Latte,Venti,4.65
6,Caramel Macchiato,Tall,3.75
7,Caramel Macchiato,Grande,4.45
8,Caramel Macchiato,Venti,4.75


In [4]:
import pandas as pd
menu = pd.read_csv("data/starbucks-menu-nutrition-drinks.csv")
menu

,name,calories,fat_g,carb_g,fiber_g,protein,sodium
0,Cool Lime Starbucks Refreshers™ Beverage,45,0,11,0,0,10
1,Ombré Pink Drink,-,-,-,-,-,-
2,Pink Drink,-,-,-,-,-,-
3,Strawberry Acai Starbucks Refreshers™ Beverage,80,0,18,1,0,10
4,Very Berry Hibiscus Starbucks Refreshers™ Beve...,60,0,14,1,0,10
...,...,...,...,...,...,...,...
172,Chocolate Smoothie,320,5,53,8,20,170
173,Strawberry Smoothie,300,2,60,7,16,130
174,Ginger Ale,-,-,-,-,-,-
175,Lemon Ale,-,-,-,-,-,-


In [5]:
food_table.merge(menu, left_on='name', right_on='name')

,name,size,price,calories,fat_g,carb_g,fiber_g,protein,sodium
0,White Chocolate Mocha,Tall,3.75,360,11,53,0,14,240
1,White Chocolate Mocha,Grande,4.45,360,11,53,0,14,240
2,White Chocolate Mocha,Venti,4.75,360,11,53,0,14,240
3,Cinnamon Dolce Latte,Tall,3.65,260,6,40,0,11,150
4,Cinnamon Dolce Latte,Grande,4.25,260,6,40,0,11,150
5,Cinnamon Dolce Latte,Venti,4.65,260,6,40,0,11,150
6,Caramel Macchiato,Tall,3.75,250,7,35,0,10,150
7,Caramel Macchiato,Grande,4.45,250,7,35,0,10,150
8,Caramel Macchiato,Venti,4.75,250,7,35,0,10,150


In [6]:
import json
import os

from kafka import KafkaConsumer, KafkaProducer
import json
import os

from kafka import KafkaConsumer, KafkaProducer


TRANSACTIONS_TOPIC = "queueing.transactions"
KAFKA_BROKER_URL = "kafka:29092"


def is_suspicious(transaction: dict) -> bool:
    """Determine whether a transaction is suspicious."""
    return transaction['amount'] >= 900



consumer = KafkaConsumer(
    TRANSACTIONS_TOPIC,
    bootstrap_servers=KAFKA_BROKER_URL,
    group_id="test",
    enable_auto_commit=False,
    auto_offset_reset="earliest"    
)

print(consumer.topics())
for message in consumer:
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,
                                          message.offset, message.key,
                                          message.value))
    break


{'_confluent-command', '_confluent-metrics', 'queueing.transactions'}
queueing.transactions:0:0: key=None value=b'{"name_employer": "Cassandra", "id_employer": 3, "name_client": "Anne Sanders", "transaction_id": "6712763899169", "payment": "cash", "timestamp": "2020-08-01 13:04:05", "product_name": "Caramel Macchiato", "product_size": "Grande", "product_price": 4.45, "percent_discount": 10}'
